# FATHOM 3 pre-processing
## TILES MOSAICING SCRIPT

The python script below looks into subfolders of specified directory and merge together all tiff files found, naming the resulting tiff as the folder from which they are generated.

- Specify the directory (`year` or `SSP scenario`) that contains the RP subfolders - not the RP subfolder itself (1in`YEARS`)!
- The output is created in the parent directory of each RP subfolder.
- The output files (1in`YEARS`.tif) will be ready for use in the script input data folder. Keep the folder structure as indicated:<br>
  `data/HZD/Country_ISO/type/year/(SSP scenario)/1inYEARS.tif`
- GDAL library must be installed in your python environment (see [readme](./README.md))
- Output is exported in the same raster format of the original tiles
- The scripts makes use of parallel processing for quicker results, so it works much better on multi-core CPUs.

### Run cell below and use the GUI, or use the manual run on the bottom cell

In [1]:
import os
import concurrent.futures
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output
from merge_utils import merge_tifs
import tkinter as tk
from tkinter import filedialog

def select_folder():
    root = tk.Tk()
    root.attributes('-topmost', True)  # This should make the window appear on top
    root.geometry("1x1+0+0")  # Make the window as small as possible
    root.withdraw()  # Hide the small window
    
    root.after(0, lambda: root.focus_force())  # Force focus on the hidden Tkinter window
    folder_selected = filedialog.askdirectory(master=root)  # Open the dialog
    
    root.destroy()  # Destroy the temporary window
    return folder_selected

class TilesMosaicingTool:
    def __init__(self):
        self.directory_input = widgets.Text(
            value='',
            placeholder='Enter the directory path',
            description='Directory:',
            disabled=False,
            layout=widgets.Layout(width='500px')
        )
        
        self.search_button = widgets.Button(
            description='Search Folder',
            disabled=False,
            button_style='',
            tooltip='Click to search for a folder',
            layout=widgets.Layout(width='120px')
        )
        
        self.process_button = widgets.Button(
            description='Process Tiles',
            disabled=False,
            button_style='',
            tooltip='Click to start processing',
            layout=widgets.Layout(width='120px')
        )
        
        self.delete_checkbox = widgets.Checkbox(
            value=False,
            description='Delete original subfolders after processing',
            disabled=False,
            indent=False,
            layout=widgets.Layout(width='500px')
        )
        
        self.output = widgets.Output()
        
        self.search_button.on_click(self.on_search_button_clicked)
        self.process_button.on_click(self.on_process_button_clicked)
        
        # Layout
        display(widgets.VBox([
            widgets.HBox([self.directory_input, self.search_button]),
            widgets.HBox([self.process_button, self.delete_checkbox]),
            self.output
        ]))
    
    def on_search_button_clicked(self, b):
        folder_path = select_folder()
        if folder_path:
            self.directory_input.value = folder_path
    
    def on_process_button_clicked(self, b):
        self.output.clear_output()
        with self.output:
            directory = self.directory_input.value
            if not directory or not os.path.isdir(directory):
                print(f"Error: {directory} is not a valid directory.")
                return
            
            print(f"Output files will be saved in: {directory}")
            
            subdirs = [os.path.join(directory, subdir) for subdir in os.listdir(directory) 
                       if os.path.isdir(os.path.join(directory, subdir))]
            
            with concurrent.futures.ProcessPoolExecutor() as executor:
                executor.map(merge_tifs, subdirs)
            
            print("Processing complete. Check the parent directory of each RP subfolder for the merged .tif files.")
            
            if self.delete_checkbox.value:
                for subdir in subdirs:
                    try:
                        shutil.rmtree(subdir)
                        print(f"Deleted: {subdir}")
                    except Exception as e:
                        print(f"Error deleting {subdir}: {e}")
                print("Original subfolders have been deleted.")
            else:
                print("Original subfolders have been kept.")
            
            print("Pre-processing completed.")

# Create and display the tool
tool = TilesMosaicingTool()

# MANUAL RUN

In [3]:
import os
import concurrent.futures
import shutil
from merge_utils import merge_tifs

# Specify the directory containing the downloaded Fathom data (tiles) 
directory = 'C:/YourWorkDirectory/Fathom3/PL/2020/'

print(f"Output files will be saved in: {directory}")

# Iterate through each subdirectory
subdirs = [os.path.join(directory, subdir) for subdir in os.listdir(directory) if os.path.isdir(os.path.join(directory, subdir))]

# Process subdirectories in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(merge_tifs, subdirs)

print("Processing complete. Check the parent directory of each RP subfolder for the merged .tif files.")

# Ask user if they want to delete the original subfolders
user_input = input("Do you want to delete the original subfolders containing the tiles? (Y/N): ").strip().lower()

if user_input == 'y':
    for subdir in subdirs:
        try:
            shutil.rmtree(subdir)
            print(f"Deleted: {subdir}")
        except Exception as e:
            print(f"Error deleting {subdir}: {e}")
    print("Original subfolders have been deleted.")
else:
    print("Original subfolders have been kept.")

print("Pre-processing completed.")

Output files will be saved in: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020
Processing complete. Check the parent directory of each RP subfolder for the merged .tif files.
Do you want to delete the original subfolders containing the tiles? (Y/N): y
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in10
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in100
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in1000
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in20
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in200
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in5
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in50
Deleted: X:/Work/WB/Risk_tools/script/dev/data/HZD/JAM/FLUVIAL_UNDEFENDED/2020\1in500
Original subfolders have been deleted.
Pre